In [1]:
using LinearAlgebra, Symbolics

In [2]:
@variables r s;

In [3]:
ν = 0.3 # Coeficiente de Poisson
EM = 2.1E06 # Módulo de Young - kgf/cm^2
C = (EM/(1-ν^2)) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]; # Matriz de constantes elásticas (Tensión plana)

In [40]:
NodalCoords = [0 0; 3 0; 1.5 3; 1.5 0; 2.25 1.5; 0.75 1.5]; # Coordenadas Globales para elemento de 6 nodos

In [5]:
# Funciones de interpolación para un elemento isoparamétrico triangular de 6 nodos
h4 = 4*r*(1-r-s)
h5 = 4*r*s
h6 = 4*s*(1-r-s)
h1 = 1-r-s-1/2*h4-1/2*h6
h2 = r-1/2*h4-1/2*h5
h3 = s-1/2*h5-1/2*h6;

In [6]:
Dr = Differential(r)
Ds = Differential(s)

(::Differential) (generic function with 3 methods)

In [7]:
# Funciones de interpolación para un elemento isoparamétrico triangular de 3 nodos
H = [h1 h2 h3 h4 h5 h6]
∇H = [transpose([expand_derivatives.(Dr(H[i])) for i in 1:6]); transpose([expand_derivatives.(Ds(H[i])) for i in 1:6])]

2×6 Matrix{Num}:
 -1 + 2.0r - 2.0(1 - r - s) + 2.0s  …  4s                -4s
 -1 + 2.0r - 2.0(1 - r - s) + 2.0s     4r  4(1 - r - s) - 4s

In [8]:
x = expand((H * NodalCoords[:,1])[1])
y = expand((H * NodalCoords[:,2])[1]);

In [39]:
# J - Jacobiano. Fundamentals of FEA, Koutromanos-pag.240 (ec.8.5.25)
J = expand.(∇H * NodalCoords) 

J_inv = inv(J) # Inversa del Jacobiano
J_det = det(J) # Determinante del Jacobiano
display(J)
display(J_inv)
display(J_det)

2×2 Matrix{Num}:
 3.0  0
 1.5  3.0

2×2 Matrix{Num}:
  0.333333  0.0
 -0.166667  0.333333

9.0

In [10]:
B1 = expand.( J_inv * ∇H)

B = expand.([B1[1,1] 0 B1[1,2] 0 B1[1,3] 0 B1[1,4] 0 B1[1,5] 0 B1[1,6] 0
    0 B1[2,1] 0 B1[2,2] 0 B1[2,3] 0 B1[2,4] 0 B1[2,5] 0 B1[2,6]
    B1[2,1] B1[1,1] B1[2,2] B1[1,2] B1[2,3] B1[1,3] B1[2,4] B1[1,4] B1[2,5] B1[1,5] B1[2,6] B1[1,6]])

3×12 Matrix{Num}:
 -1.0 + 1.33333r + 1.33333s    …   0
  0                                1.33333 - 1.33333r - 2.0s
 -0.5 + 0.666667r + 0.666667s     -1.33333s

In [38]:
#Integramos por medio de cuadratura de Gauss para siete puntos de integración sobre dominios triangulares
r1 = 0.1012865073235
r2 = 0.7974269853531
r3 = r1
r4 = 0.4701420641051
r5 = r4
r6 = 0.0597158717898
r7 =  1/3

s1 = r1
s2 = r1
s3 = r2
s4 = r6
s5 = r4
s6 = r4
s7 = r7

w1 = 0.1259391805448
w2 = w1
w3 = w1
w4 = 0.1323941527885
w5 = w4
w6 = w4
w7 = 0.225

GL7T = [r1 s1; r2 s2; r3 s3; r4 s4; r5 s5; r6 s6; r7 s7] # Puntos de Gauss para integración
WGL7T = [w1 w2 w3 w4 w5 w6 w7]; # Pesos para  puntos de Gauss


In [12]:
K = zeros(12,12) 

for i in 1:7
    Jg = substitute(J, Dict([r => GL7T[i,1], s => GL7T[i,2]]))        
    Jg_inv = substitute(J_inv, Dict([r => GL7T[i,1], s => GL7T[i,2]]))
    Jg_det = substitute(J_det, Dict([r => GL7T[i,1], s => GL7T[i,2]]))                
    Bg = substitute(B, Dict([r => GL7T[i,1], s => GL7T[i,2]]))
    
    K += 1/2 * (transpose(Bg) * C * Bg * Jg_det * WGL7T[i])
        
end
display(K)

12×12 Matrix{Num}:
      1.25481e6   375000.0         …  -2.69231e5   -4.61538e5
 375000.0              6.92308e5      -5.38462e5   -7.69231e5
      3.50962e5    -9615.38           -2.03756e-7   4.65079e-8
   9615.38         38461.5             4.36194e-8  -8.89995e-8
  67307.7              1.34615e5      -2.69231e5   -5.38462e5
      1.15385e5        1.92308e5   …  -4.61538e5   -7.69231e5
     -1.40385e6    38461.5            -5.38462e5    1.0e6
 -38461.5             -1.53846e5       1.0e6       -1.53846e6
      3.83588e-8       1.14233e-8     -2.80769e6   -8.36226e-8
      1.14596e-8       2.11439e-8     -6.75136e-8  -3.07692e5
     -2.69231e5       -5.38462e5   …   3.88462e6   -2.90165e-8
     -4.61538e5       -7.69231e5      -2.90165e-8   3.38462e6

In [13]:
 # Imponemos condiciones de contorno de Dirichlet
 # U1y=0 => U2=0
    K[2,:] .= 0
    K[:,2] .= 0
    K[2,2] =1
 # U2x=0 => U3=0
    K[3,:] .= 0
    K[:,3] .= 0
    K[3,3] =1
 # U2y=0 => U4=0
    K[4,:] .= 0
    K[:,4] .= 0
    K[4,4] =1

display(K)

12×12 Matrix{Num}:
      1.25481e6   0  0  0  67307.7         …  -2.69231e5   -4.61538e5
      0           1  0  0      0               0            0
      0           0  1  0      0               0            0
      0           0  0  1      0               0            0
  67307.7         0  0  0      4.03846e5      -2.69231e5   -5.38462e5
      1.15385e5   0  0  0      0.0         …  -4.61538e5   -7.69231e5
     -1.40385e6   0  0  0     -5.86733e-8     -5.38462e5    1.0e6
 -38461.5         0  0  0     -2.02999e-8      1.0e6       -1.53846e6
      3.83588e-8  0  0  0     -2.69231e5      -2.80769e6   -8.36226e-8
      1.14596e-8  0  0  0      5.38462e5      -6.75136e-8  -3.07692e5
     -2.69231e5   0  0  0     -2.69231e5   …   3.88462e6   -2.90165e-8
     -4.61538e5   0  0  0     -5.38462e5      -2.90165e-8   3.38462e6

In [14]:
#= \(A,B)
Matrix division using a polyalgorithm. For input matrices A and B, the result X is such that A*X == B when A is square. 
The solver that is used depends upon the structure of A. If A is upper or lower triangular (or diagonal), no factorization 
of A is required and the system is solved with either forward or backward substitution. For non-triangular square matrices, 
an LU factorization is used.
=#

In [31]:
# CASO 1. F1 = 500kgf ; F2 = 0kgf

R1 =[0, 0, 0, 0, 500., 0, 0, 0, 0, 0, 0, 0] # Fuerzas nodales en kgf.
U1 = K \ R1 # U1 --> Desplazamientos nodales.
display(U1)
E1 = B * U1 # E1 --> vector de deformaciones caso de carga 1. Voigt notation
σ1 = C * E1 # σ1 --> vector de esfuerzos, caso de carga 1.

# Matriz de deformaciones y de esfuerzos nodales
Enodal1 = zeros(Num,3,6)
σNodal1 = zeros(Num,3,6)

ParamCoords = [0 0; 1 0; 0 1; 1/2 0; 1/2 1/2; 0 1/2]

for i in 1:6, j in 1:3
    Enodal1[j,i] = substitute(E1[j], Dict([r => ParamCoords[i,1], s => ParamCoords[i,2]]))
    if (abs(Enodal1[j,i]) <= 1E-8)  Enodal1[j,i] = 0 end
    σNodal1[j,i] = substitute(σ1[j], Dict([r => ParamCoords[i,1], s => ParamCoords[i,2]]))
    if (abs(σNodal1[j,i]) <= 1E-8)  σNodal1[j,i] = 0 end
end

display(Enodal1)
display(σNodal1)

#=
# Deformaciones en los puntos de Gauss.
EGauss1 = zeros(Num,3,7)
SigmaGauss1 = zeros(Num,3,7)

for i in 1:7, j in 1:3
    EGauss1[j,i] = substitute(E1[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
    SigmaGauss1[j,i] = substitute(σ1[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
end
display(EGauss1)
display(SigmaGauss1)
=#

12-element Vector{Num}:
  0.0007142857142859171
  0.0
  0.0
  0.0
  0.004821428571429651
  0.00017857142857164574
  0.0003214285714286487
  0.00017857142857155732
  0.0014553571428574355
 -0.00047321428571432537
  0.0014553571428574195
  0.0008482142857146421

3×6 Matrix{Num}:
 -0.000285714  -0.000190476   0.000238095  …   2.38095e-5   -2.38095e-5
  0.000952381  -0.000809524  -7.14286e-5      -0.000440476   0.000440476
  0             0             0.0012381        0.000619048   0.000619048

3×6 Matrix{Num}:
    0    -1000.0   500.0  -500.0   -250.0   250.0
 2000.0  -2000.0     0       0    -1000.0  1000.0
    0        0    1000.0     0      500.0   500.0

In [37]:
sum(Enodal1[3,:]) / length(Enodal1[3,:])

0.00041269841269847026

In [27]:
sum(σNodal1[3,:]) / length(σNodal1[3,:])

333.3333333333798

In [16]:
# CASO 2. F1 = 0kgf ; F2 = 500kgf

R2 =[0, 0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0] # Fuerzas nodales en kgf.
U2 = K \ R2 # U3 --> Desplazamientos nodales.
E2 = B * U2
σ2 = C * E2

# Matriz de deformaciones nodales
σNodal2 = zeros(Num,3,6)
Enodal2 = zeros(Num,3,6)
CoordEl = [0 0; 1 0; 0 1; 1/2 0; 1/2 1/2; 0 1/2]

for i in 1:6, j in 1:3
    Enodal2[j,i] = substitute(E2[j], Dict(r => CoordEl[i,1], s => CoordEl[i,2]),)
    if (abs(Enodal2[j,i]) <= 1E-8)  Enodal2[j,i] = 0 end
    σNodal2[j,i] = substitute(σ2[j], Dict(r => CoordEl[i,1], s => CoordEl[i,2]),)
    if (abs(σNodal2[j,i]) <= 1E-8)  σNodal2[j,i] = 0 end
end
display(Enodal2)
display(σNodal2)

#=
# Deformaciones en los puntos de Gauss.
#Caso 2.
EGauss2 = zeros(Num,3,7)
SigmaGauss2 = zeros(Num,3,7)

for i in 1:7, j in 1:3
    EGauss2[j,i] = substitute(E2[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
    SigmaGauss2[j,i] = substitute(σ2[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
end
display(EGauss2)
display(SigmaGauss2)
=#

3×6 Matrix{Num}:
 -0.000119048  -0.000119048  9.52381e-5   …  -1.19048e-5   -1.19048e-5
  3.57143e-5    3.57143e-5   0.000404762      0.000220238   0.000220238
  0.000619048  -0.000619048  0               -0.000309524   0.000309524

3×6 Matrix{Num}:
 -250.0  -250.0   500.0  -250.0   125.0  125.0
    0       0    1000.0     0     500.0  500.0
  500.0  -500.0     0       0    -250.0  250.0

In [17]:
# CASO 3. F1 = 500kgf ; F2 = 500kgf

R3 =[0, 0, 0, 0, 500, 500, 0, 0, 0, 0, 0, 0] # Fuerzas nodales en kgf.
U3 = K \ R3 # U3 --> Desplazamientos nodales.
E3 = B * U3
σ3 = C * E3

# Matriz de deformaciones nodales
Enodal3 = zeros(Num,3,6)
σNodal3 = zeros(Num,3,6)
CoordEl = [0 0; 1 0; 0 1; 1/2 0; 1/2 1/2; 0 1/2]

for i in 1:6, j in 1:3
    Enodal3[j,i] = substitute(E3[j], Dict(r => CoordEl[i,1], s => CoordEl[i,2]),)
    if (abs(Enodal3[j,i]) <= 1E-8)  Enodal3[j,i] = 0 end
    σNodal3[j,i] = substitute(σ3[j], Dict(r => CoordEl[i,1], s => CoordEl[i,2]),)
    if (abs(σNodal3[j,i]) <= 1E-8)  σNodal3[j,i] = 0 end
end
display(Enodal3)
display(σNodal3)

#=
# Deformaciones en los puntos de Gauss.
#Caso 3.
EGauss3 = zeros(Num,3,7)
SigmaGauss3 = zeros(Num,3,7)

for i in 1:7, j in 1:3
    EGauss3[j,i] = substitute(E3[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
    SigmaGauss3[j,i] = substitute(σ3[j], Dict(r => GL7T[i,1], s => GL7T[i,2]),)
end
display(EGauss3)
display(SigmaGauss3)
=#

3×6 Matrix{Num}:
 -0.000404762  -0.000309524  0.000333333  …   1.19048e-5   -3.57143e-5
  0.000988095  -0.00077381   0.000333333     -0.000220238   0.000660714
  0.000619048  -0.000619048  0.0012381        0.000309524   0.000928571

3×6 Matrix{Num}:
 -250.0  -1250.0  1000.0  -750.0  -125.0   375.0
 2000.0  -2000.0  1000.0     0    -500.0  1500.0
  500.0   -500.0  1000.0     0     250.0   750.0